In [ ]:
import os, sys
sys.path.insert(0, '../src')

In [ ]:
print(os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), '..', 'src')))

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib widget

In [ ]:
from graphrank.graphrank import GraphRank
from graphrank.utils import GraphUtils, TextPreprocess
from graphrank import dgraph as dg
import networkx as nx
import json as js

In [ ]:
from scipy.spatial.distance import cosine

In [ ]:
gr = GraphRank()
tp = TextPreprocess()
utils = GraphUtils()

In [ ]:
def read_json(json_file):
    with open(json_file) as f:
        article = js.load(f)
    return article

In [ ]:
article = read_json('embed_hr_json.json')

## Build graph from sentences

In [ ]:
g = nx.Graph()

In [ ]:
for i, articles in enumerate(article):
    sent_id = articles['sent_id']
    sentence = articles['sentence']
    embed = articles['embedding_list']
    if sent_id not in g.nodes():
        g.add_node(sent_id, text=sentence, embed_list=embed)
    for j in range(i + 1, len(article)):
        neighbor_s_id = article[j]['sent_id']
        neighbor_sentece = article[j]['sentence']
        neighbor_embed = article[j]['embedding_list']
        similarity_score = cosine(embed, neighbor_embed)
        if neighbor_s_id not in g.nodes():
            g.add_node(neighbor_s_id, text=neighbor_sentece, embed_list=neighbor_embed)
        g.add_edge(sent_id, neighbor_s_id, sim_score=similarity_score)

In [ ]:
list(g.edges.data())

## Run PageRank on default weights (w=1)

In [ ]:
pg_rank_def = nx.pagerank_scipy(g, weight=None)

In [ ]:
pg_rank_def

## Run PageRank on similarity scores

In [ ]:
pg_rank_sim = nx.pagerank_scipy(g, weight='sim_score')

In [ ]:
pg_rank_sim

In [ ]:
def sort_by_value(item_list, order='desc'):
    """
    A utility function to sort lists by their value.
    Args:
        item_list:
        order:

    Returns:

    """

    if order == 'desc':
        sorted_list = sorted(item_list, key=lambda x: (x[1], x[0]), reverse=True)
    else:
        sorted_list = sorted(item_list, key=lambda x: (x[1], x[0]), reverse=False)

    return sorted_list

In [ ]:
sorted_rank = dict(sort_by_value(pg_rank_sim.items()))

In [ ]:
g.nodes.data()[0]

In [ ]:
for k, v in sorted_rank.items():
    print(k, g.nodes.data()[k].get('text'))
    print(v)
    print()

In [ ]:
bet = nx.degree_centrality(g)

In [ ]:
bet